In [6]:
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'

import torch
from transformers import LlamaTokenizer
from pefty_llama.modeling import create_model

In [1]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = create_model("7b", hf_path="../model_checkpoints/llama-7b-hf")

/mnt/shared_home/vlialin/miniconda3/envs/pefty_llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin /mnt/shared_home/vlialin/miniconda3/envs/pefty_llama/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
100%|██████████| 33/33 [00:07<00:00,  4.38it/s]


In [2]:
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_trainable_params:,}")

Total trainable parameters: 6,738,415,616


In [3]:
input_ids = tokenizer("42 is the answer", return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")
out1 = model.generate(input_ids, generation_length=10)
print(tokenizer.decode(out1[0]))

 ⁇  42 is the answer first of its own kind.
The 2


In [4]:
from pefty_llama.peft.ia3 import IA3
model = IA3(model).to("cuda")

In [7]:
out2 = model.generate(input_ids, generation_length=10)
print(tokenizer.decode(out2[0]))
assert torch.all(out1 == out2), "At initialization, the model should produce the same output as the original model."

 ⁇  42 is the answer first of its own kind.
The 2


In [8]:
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_trainable_params:,}")

Total trainable parameters: 614,400


In [5]:
model

IA3(
  (base_model): LLaMAModel(
    (model): LLaMAInnerModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LLaMALayer(
          (self_attn): IA3Attention(
            (q_proj): NoInitLinear(in_features=4096, out_features=4096, bias=False)
            (k_proj): NoInitLinear(in_features=4096, out_features=4096, bias=False)
            (v_proj): NoInitLinear(in_features=4096, out_features=4096, bias=False)
            (o_proj): NoInitLinear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): RotaryEmbedding()
          )
          (mlp): IA3MLP(
            (gate_proj): NoInitLinear(in_features=4096, out_features=11008, bias=False)
            (up_proj): NoInitLinear(in_features=4096, out_features=11008, bias=False)
            (down_proj): NoInitLinear(in_features=11008, out_features=4096, bias=False)
          )
          (input_layernorm): RMSNorm()
          (post_attention_layernorm): RMSNorm()
        )
      